In [1]:
import pandas as pd

In [2]:
data = pd.read_csv("translated_data.csv")

In [3]:
data.head()

,text,summary,machine_summary,Актуальность,Последовательность,Беглость,Согласованность
0,(CNN) В прошлом году расистские высказывания Д...,"Стивиано должен вернуть 2,6 миллиона долларов ...","Дональд Стерлинг, игрок прошлогодней команды Н...",1.666667,1.000000,1.000000,1.333333
1,(CNN) В прошлом году расистские высказывания Д...,"Стивиано должен вернуть 2,6 миллиона долларов ...","Дональд Стерлинг обвинил Стивиано в том, что о...",1.666667,2.333333,4.666667,3.000000
2,(CNN) В прошлом году расистские высказывания Д...,"Стивиано должен вернуть 2,6 миллиона долларов ...",судья Лос-Анджелеса обязал В. Стивиано выплати...,2.333333,4.666667,4.333333,1.000000
3,(CNN) В прошлом году расистские высказывания Д...,"Стивиано должен вернуть 2,6 миллиона долларов ...",Жена Дональда Стерлинга подала в суд на Стивиа...,3.333333,5.000000,4.333333,2.666667
4,(CNN) В прошлом году расистские высказывания Д...,"Стивиано должен вернуть 2,6 миллиона долларов ...",расистские высказывания Дональда Стерлинга в п...,3.000000,5.000000,5.000000,4.666667


## Metrics

In [ ]:
results = {}

### BERTScore

In [4]:
from evaluate import load
bertscore = load("bertscore")
references = list(data["summary"])
predictions = list(data["machine_summary"])
results["bertscore"] = bertscore.compute(predictions=predictions, references=references, lang="ru", verbose=True, model_type="microsoft/mdeberta-v3-base", nthreads=16, batch_size=16, use_fast_tokenizer=False)

calculating scores...
computing bert embedding.


  0%|          | 0/102 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/100 [00:00<?, ?it/s]

done in 56907.13 seconds, 0.03 sentences/sec


In [8]:
print(len(results["bertscore"]["f1"]))

1600


In [9]:
data["bertscore_f1"] = results["bertscore"]["f1"]

In [10]:
data.head()

,text,summary,machine_summary,Актуальность,Последовательность,Беглость,Согласованность,bertscore_f1
0,(CNN) В прошлом году расистские высказывания Д...,"Стивиано должен вернуть 2,6 миллиона долларов ...","Дональд Стерлинг, игрок прошлогодней команды Н...",1.666667,1.000000,1.000000,1.333333,0.688328
1,(CNN) В прошлом году расистские высказывания Д...,"Стивиано должен вернуть 2,6 миллиона долларов ...","Дональд Стерлинг обвинил Стивиано в том, что о...",1.666667,2.333333,4.666667,3.000000,0.666818
2,(CNN) В прошлом году расистские высказывания Д...,"Стивиано должен вернуть 2,6 миллиона долларов ...",судья Лос-Анджелеса обязал В. Стивиано выплати...,2.333333,4.666667,4.333333,1.000000,0.650881
3,(CNN) В прошлом году расистские высказывания Д...,"Стивиано должен вернуть 2,6 миллиона долларов ...",Жена Дональда Стерлинга подала в суд на Стивиа...,3.333333,5.000000,4.333333,2.666667,0.684118
4,(CNN) В прошлом году расистские высказывания Д...,"Стивиано должен вернуть 2,6 миллиона долларов ...",расистские высказывания Дональда Стерлинга в п...,3.000000,5.000000,5.000000,4.666667,0.676089


### BLEU

In [11]:
import razdel
from evaluate import load
from tqdm import tqdm
# import nltk

bleu = load('bleu')

def a(x):
    tokens = list(razdel.tokenize(x))
    return [_.text for _ in tokens]

references = list(data["summary"])
predictions = list(data["machine_summary"])
results["bleu"] = []
for ref, pred in tqdm(zip(references, predictions)):
    bl = bleu.compute(predictions=[pred], references=[ref], tokenizer=a, smooth=True)
    # bl = nltk.translate.bleu(pred, ref)
    results["bleu"].append(bl["bleu"])


1600it [00:02, 566.87it/s]


In [12]:
print(len(results["bleu"]))

1600


In [13]:
data["bleu"] = results["bleu"]

In [14]:
data.head()

,text,summary,machine_summary,Актуальность,Последовательность,Беглость,Согласованность,bertscore_f1,bleu
0,(CNN) В прошлом году расистские высказывания Д...,"Стивиано должен вернуть 2,6 миллиона долларов ...","Дональд Стерлинг, игрок прошлогодней команды Н...",1.666667,1.000000,1.000000,1.333333,0.688328,0.101056
1,(CNN) В прошлом году расистские высказывания Д...,"Стивиано должен вернуть 2,6 миллиона долларов ...","Дональд Стерлинг обвинил Стивиано в том, что о...",1.666667,2.333333,4.666667,3.000000,0.666818,0.055018
2,(CNN) В прошлом году расистские высказывания Д...,"Стивиано должен вернуть 2,6 миллиона долларов ...",судья Лос-Анджелеса обязал В. Стивиано выплати...,2.333333,4.666667,4.333333,1.000000,0.650881,0.080116
3,(CNN) В прошлом году расистские высказывания Д...,"Стивиано должен вернуть 2,6 миллиона долларов ...",Жена Дональда Стерлинга подала в суд на Стивиа...,3.333333,5.000000,4.333333,2.666667,0.684118,0.060453
4,(CNN) В прошлом году расистские высказывания Д...,"Стивиано должен вернуть 2,6 миллиона долларов ...",расистские высказывания Дональда Стерлинга в п...,3.000000,5.000000,5.000000,4.666667,0.676089,0.109052


### ROUGE

In [15]:
import razdel
from evaluate import load

rouge = load('rouge')
references = list(data["summary"])
predictions = list(data["machine_summary"])

def a(x):
    tokens = list(razdel.tokenize(x))
    return [_.text for _ in tokens]

results["rouge"] = rouge.compute(predictions=predictions, references=references, use_aggregator=False, tokenizer=a)

In [17]:
print(results["rouge"].keys())

dict_keys(['rouge1', 'rouge2', 'rougeL', 'rougeLsum'])


In [18]:
print(len(results["rouge"]["rouge1"]))

1600


In [19]:
data["rouge1"] = results["rouge"]["rouge1"]

In [20]:
data.head()

,text,summary,machine_summary,Актуальность,Последовательность,Беглость,Согласованность,bertscore_f1,bleu,rouge1
0,(CNN) В прошлом году расистские высказывания Д...,"Стивиано должен вернуть 2,6 миллиона долларов ...","Дональд Стерлинг, игрок прошлогодней команды Н...",1.666667,1.000000,1.000000,1.333333,0.688328,0.101056,0.300885
1,(CNN) В прошлом году расистские высказывания Д...,"Стивиано должен вернуть 2,6 миллиона долларов ...","Дональд Стерлинг обвинил Стивиано в том, что о...",1.666667,2.333333,4.666667,3.000000,0.666818,0.055018,0.297872
2,(CNN) В прошлом году расистские высказывания Д...,"Стивиано должен вернуть 2,6 миллиона долларов ...",судья Лос-Анджелеса обязал В. Стивиано выплати...,2.333333,4.666667,4.333333,1.000000,0.650881,0.080116,0.285714
3,(CNN) В прошлом году расистские высказывания Д...,"Стивиано должен вернуть 2,6 миллиона долларов ...",Жена Дональда Стерлинга подала в суд на Стивиа...,3.333333,5.000000,4.333333,2.666667,0.684118,0.060453,0.329897
4,(CNN) В прошлом году расистские высказывания Д...,"Стивиано должен вернуть 2,6 миллиона долларов ...",расистские высказывания Дональда Стерлинга в п...,3.000000,5.000000,5.000000,4.666667,0.676089,0.109052,0.297297


### METEOR

In [24]:
from tqdm import tqdm
from threading import Thread
from evaluate import load
import os, contextlib

meteor = load('meteor')

references = list(data["summary"])
predictions = list(data["machine_summary"])

results["meteor"] = []
for pred, ref in zip(predictions, references):
    results["meteor"].append(meteor.compute(predictions=[pred], references=[ref])["meteor"])
#results["meteor"].append(meteor.compute(predictions=[predictions], references=[references]))



[nltk_data] Downloading package wordnet to /home/skatori/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /home/skatori/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/skatori/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [26]:
print(len(results["meteor"]))

1600


In [27]:
data["meteor"] = results["meteor"]

In [28]:
data.head()

,text,summary,machine_summary,Актуальность,Последовательность,Беглость,Согласованность,bertscore_f1,bleu,rouge1,meteor
0,(CNN) В прошлом году расистские высказывания Д...,"Стивиано должен вернуть 2,6 миллиона долларов ...","Дональд Стерлинг, игрок прошлогодней команды Н...",1.666667,1.000000,1.000000,1.333333,0.688328,0.101056,0.300885,0.270348
1,(CNN) В прошлом году расистские высказывания Д...,"Стивиано должен вернуть 2,6 миллиона долларов ...","Дональд Стерлинг обвинил Стивиано в том, что о...",1.666667,2.333333,4.666667,3.000000,0.666818,0.055018,0.297872,0.352888
2,(CNN) В прошлом году расистские высказывания Д...,"Стивиано должен вернуть 2,6 миллиона долларов ...",судья Лос-Анджелеса обязал В. Стивиано выплати...,2.333333,4.666667,4.333333,1.000000,0.650881,0.080116,0.285714,0.214511
3,(CNN) В прошлом году расистские высказывания Д...,"Стивиано должен вернуть 2,6 миллиона долларов ...",Жена Дональда Стерлинга подала в суд на Стивиа...,3.333333,5.000000,4.333333,2.666667,0.684118,0.060453,0.329897,0.370017
4,(CNN) В прошлом году расистские высказывания Д...,"Стивиано должен вернуть 2,6 миллиона долларов ...",расистские высказывания Дональда Стерлинга в п...,3.000000,5.000000,5.000000,4.666667,0.676089,0.109052,0.297297,0.226064


## Export

In [29]:
data.to_csv("data_and_metrics.csv", index=False)